In [46]:
import cvxpy as cp
import numpy as np
from cvxpy.reductions.dcp2cone.cone_matrix_stuffing import ConeMatrixStuffing
from cvxpy.perspective.perspective_utils import form_cone_constraint

In [47]:
x = cp.Variable()

obj = cp.square(x)+2*x-4
constraints = []
# constraints = [x <= 1, x >= .5]

prob = cp.Problem(cp.Minimize(obj),constraints)

# prob.solve()

In [48]:
chain = prob._construct_chain()

chain.reductions = chain.reductions[:-1] #skip solver reduction

prob_canon = chain.apply(prob)[0] #grab problem instance

In [49]:
# get cone representation of c, A, and b for some problem.
c = prob_canon.c.toarray().flatten()[:-1] # TODO: why
d = prob_canon.c.toarray().flatten()[-1]

Ab = prob_canon.A.toarray().reshape((-1,len(c)+1),order="F")
A, b = Ab[:,:-1], Ab[:,-1]

cones = prob_canon.cone_dims # get the cones used and the dims

# given f in epigraph form, aka epi f = \{(x,t) | f(x) \leq t\} 
# = \{(x,t) | Fx +tg + e \in K} for K a cone, the epigraph of the
# perspective, \{(x,s,t) | sf(x/s) \leq t} = \{(x,s,t) | Fx + tg + se \in K\}
# If I have the problem "minimize f(x)" written in the CVXPY compatible 
# "c^Tx, Ax+b \in K" form, I can re-write this in the graph form above via
# x,t \in \epi f iff Ax + b \in K and t-c^Tx \in R_+ which I can further write
# with block matrices as Fx + tg + e \in K \times R_+
# with F = [A ], g = [0], e = [b]  
#          [-c]      [1]      [-d]

# Actually, all we need is Ax + 0*t + sb \in K, -c^Tx + t - ds >= 0

# F = np.vstack([A,-c])
# g = np.hstack([np.zeros(A.shape[0]),1])
# e = np.hstack([b,d])

# Now we go to the graph form of the perspective. We add the constraint
# that s be nonnegative because we are unsure whether it is implicit.
# We want Fx + gt + se \in K \times R_+ in p^T(x,s,t), A, b form 
# A_pers = [F g e] b_pers = [0], p = (0 , 1 , 0) with variables in the order (x,t,s) = z
# The final problem is minimzie p^Tz subject to A_persp z + b \in K \times R_+

# A_pers = np.hstack([F,g.reshape((-1,1)),e.reshape((-1,1))])

# b_pers = np.zeros(A_pers.shape[0])
# p = np.zeros(A_pers.shape[1])
# p[-2] = 1

t = cp.Variable()
s = cp.Variable()
x_canon = prob_canon.x

# x_new = cp.Variable(x_canon.shape)

# z = cp.hstack([x_canon,t,s])

# constraints = [A_pers[:-2]@z \in K] # figure out general cone

constraints = []

x_pers = A@x_canon + s*b

i = 0
for con in prob_canon.constraints:
    sz, tp = con.size, type(con)
    guy = x_pers[i:i+sz]
    # print(sz)
    pers_constraint = form_cone_constraint(guy,con)
    constraints.append(pers_constraint)
    i += sz
constraints.append(s >= 0)
constraints.append(-c@x_canon + t - s*d >= 0)
constraints.append( s <= .5)

constraints.extend([x_canon[1] >= 2])

perp_prob = cp.Problem(cp.Minimize(t),constraints)

In [50]:
prob_canon.var_id_to_col

{654: 0, 648: 1}

In [51]:
[v.id for v in prob.variables()]

[648]

In [35]:
perp_prob.solve()

s.value, x_canon.value[1], perp_prob.value

# perp_prob.constraints

(array(0.5), 2.0000000001441576, 9.999999956481906)

In [45]:
xx = cp.Variable()
ss = cp.Variable()

obj = cp.quad_over_lin(xx,ss) + 2*xx - 4*ss
constraints = [xx >= 2,ss <=.5 ]

prob_val = cp.Problem(cp.Minimize(obj),constraints)

prob_val.solve()

10.000000017815246

In [37]:
ss.value,xx.value

(array(0.5), array(2.))

In [33]:
d

-4.0